# LLM Distillation Models

<div class="align-center">

<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a>
</div>

## วัตถุประสงค์การเรียนรู้


1. ทดลองใช้โมเดล distilled ระดับ production (DeepSeek-R1-Distill)



ทดลองใช้ **DeepSeek-R1-Distill-Qwen-1.5B** - โมเดล distilled ระดับ production

https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

## ติดตั้ง Dependencies

In [ ]:
%%capture
!uv pip install -q transformers accelerate bitsandbytes torch

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print("✓ Libraries imported successfully")

In [ ]:
deepseek_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

deepseek_tokenizer = AutoTokenizer.from_pretrained(deepseek_model_name)
deepseek_model = AutoModelForCausalLM.from_pretrained(
    deepseek_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"\n โหลดโมเดล {deepseek_model_name}")
print(f"  พารามิเตอร์: ~1.5B")

In [ ]:
# สร้าง pipeline
deepseek_pipe = pipeline(
    "text-generation",
    model=deepseek_model,
    tokenizer=deepseek_tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

print(" DeepSeek pipeline พร้อม")

### ทดสอบ DeepSeek กับงาน Reasoning

In [ ]:
reasoning_tests = [
    {
        "name": "Math Reasoning",
        "prompt": """Solve this problem step by step:

A store sells notebooks for $3 each and pens for $2 each. If you buy 5 notebooks and 8 pens, how much will you pay in total?

Answer:"""
    },
    {
        "name": "Logic Puzzle",
        "prompt": """Answer this logic puzzle:

If all cats are animals, and some animals are pets, can we conclude that all cats are pets?

Explain your reasoning:"""
    },
    {
        "name": "Code Generation",
        "prompt": """Write a Python function to check if a string is a palindrome.

Function:"""
    }
]

print("\n" + "="*60)
print("ทดสอบโมเดล DeepSeek distilled")
print("="*60)

for test in reasoning_tests:
    print(f"\n[{test['name']}]")
    print("-"*60)
    print(test['prompt'])
    print("\nDeepSeek Response:")
    print("-"*60)

    response = deepseek_pipe(test['prompt'], max_new_tokens=300)[0]['generated_text']
    print(response[len(test['prompt']):])
    print("="*60)

## ตรวจสอบประสิทธิภาพการใช้งาน

In [ ]:
import time

test_prompts_perf = [
    "What is the captial city of France?",
    "Explain Machine Learning in simple words",
    "Write a haiku about coding."
]

print("\n" + "="*60)
print("การทดสอบความเร็ว inference")
print("="*60)

total_time = 0
for prompt in test_prompts_perf:
    start = time.time()
    _ = deepseek_pipe(prompt, max_new_tokens=50)[0]['generated_text']
    elapsed = time.time() - start
    total_time += elapsed
    print(f"Prompt: {prompt[:40]:40s} | เวลา: {elapsed:.2f}s")

avg_time = total_time / len(test_prompts_perf)
print(f"\nเวลา inference เฉลี่ย: {avg_time:.2f}s")
print(f"ขนาดโมเดล: ~1.5B พารามิเตอร์")
print(f"การใช้หน่วยความจำ: ~3-4 GB VRAM (FP16)")